In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import cv2
import pydot

In [ ]:
#load_dataset function to load the data and resize the images to 50x50
def load_dataset(directory):
  images = []
  labels = []
  for idx, label in enumerate(uniq_labels):
    for file in os.listdir(directory + '/'+label):
      filepath = directory +'/'+ label + "/" + file
      img = cv2.resize(cv2.imread(filepath),(50,50))
      images.append(img)
      labels.append(idx)
  images = np.asarray(images)
  labels = np.asarray(labels)
  return images, labels

In [ ]:
#display_images function to show examples
def display_images(x_data,y_data, title, display_label = True):
    x, y = x_data,y_data
    fig, axes = plt.subplots(5, 8, figsize = (18, 5))
    fig.subplots_adjust(hspace = 0.5, wspace = 0.5)
    fig.suptitle(title, fontsize = 18)
    for i, ax in enumerate(axes.flat):
        ax.imshow(cv2.cvtColor(x[i], cv2.COLOR_BGR2RGB))
        if display_label:
            ax.set_xlabel(uniq_labels[y[i]])
        ax.set_xticks([])
        ax.set_yticks([])
    plt.show()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#loading_dataset into X_pre and Y_pre
data_dir = '/content/drive/My Drive/Gesture Image Data'
uniq_labels = sorted(os.listdir(data_dir))
X_pre, Y_pre = load_dataset(data_dir)
print(X_pre.shape, Y_pre.shape)

KeyboardInterrupt: 

In [ ]:
#spliting dataset into 80% train, 10% validation and 10% test data
X_train, X_test, Y_train, Y_test = train_test_split(X_pre, Y_pre, test_size = 0.8)
X_test, X_eval, Y_test, Y_eval = train_test_split(X_test, Y_test, test_size = 0.5)

In [ ]:
#print shapes and show examples for each set
print("Train images shape",X_train.shape, Y_train.shape)
print("Test images shape",X_test.shape, Y_test.shape)
print("Evaluate image shape",X_eval.shape, Y_eval.shape)
print("Printing the labels",uniq_labels, len(uniq_labels))

Train images shape (11100, 50, 50, 3) (11100,)
Test images shape (22200, 50, 50, 3) (22200,)
Evaluate image shape (22201, 50, 50, 3) (22201,)
Printing the labels ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '_'] 37


In [ ]:
# converting Y_tes and Y_train to One hot vectors using to_categorical
# example of one hot => '1' is represented as [0. 1. 0. . . . . 0.]
Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)
Y_eval = to_categorical(Y_eval)
X_train = X_train / 255.
X_test = X_test/ 255.
X_eval = X_eval/ 255.

In [ ]:
# building our model
# Define input shape here
input_shape = (50, 50, 3)
model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=input_shape), # explicitly define input shape
        tf.keras.layers.Conv2D(16, (3,3), activation ='relu'), # input_shape removed from here
        tf.keras.layers.Conv2D(16, (3,3), activation ='relu'),
        tf.keras.layers.Conv2D(16, (3,3), activation ='relu'),
        tf.keras.layers.MaxPool2D((2,2)),
        tf.keras.layers.Conv2D(32, (3,3), activation ='relu'),
        tf.keras.layers.Conv2D(32, (3,3), activation ='relu'),
        tf.keras.layers.Conv2D(32, (3,3), activation ='relu'),
        tf.keras.layers.MaxPool2D((2,2)),
        tf.keras.layers.Conv2D(64, (3,3), activation ='relu'),
        tf.keras.layers.Conv2D(64, (3,3), activation ='relu'),
        tf.keras.layers.Conv2D(64, (3,3), activation ='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(37, activation='softmax')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [ ]:
#compiling the model
#default batch size 32
#default learning rate is 0.001
model.compile(optimizer = 'adam',
               loss = 'categorical_crossentropy',
               metrics=['accuracy'],)

In [ ]:
#start training(fitting) the data
history = model.fit(X_train, Y_train, epochs=5, verbose=1,
                validation_data=(X_eval, Y_eval))

Epoch 1/5
347/347 ━━━━━━━━━━━━━━━━━━━━ 101s 282ms/step - accuracy: 0.3877 - loss: 2.1455 - val_accuracy: 0.9487 - val_loss: 0.1720
Epoch 2/5
347/347 ━━━━━━━━━━━━━━━━━━━━ 97s 280ms/step - accuracy: 0.9571 - loss: 0.1464 - val_accuracy: 0.9739 - val_loss: 0.1008
Epoch 3/5
347/347 ━━━━━━━━━━━━━━━━━━━━ 149s 300ms/step - accuracy: 0.9817 - loss: 0.0643 - val_accuracy: 0.9709 - val_loss: 0.0862
Epoch 4/5
347/347 ━━━━━━━━━━━━━━━━━━━━ 132s 271ms/step - accuracy: 0.9817 - loss: 0.0573 - val_accuracy: 0.9773 - val_loss: 0.0738
Epoch 5/5
347/347 ━━━━━━━━━━━━━━━━━━━━ 98s 282ms/step - accuracy: 0.9913 - loss: 0.0275 - val_accuracy: 0.9747 - val_loss: 0.0734


In [ ]:
#testing
model.evaluate(X_test, Y_test)

694/694 ━━━━━━━━━━━━━━━━━━━━ 35s 50ms/step - accuracy: 0.9761 - loss: 0.0701


[0.06833533197641373, 0.9761261343955994]

In [ ]:
#Export the model to ONNX format
!pip install tf2onnx
import tf2onnx
import onnx

In [ ]:
# Convert the model to ONNX
model.export("ResNet18.onnx",format="onnx")

Saved artifact at 'ResNet18.onnx'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 50, 50, 3), dtype=tf.float32, name='keras_tensor_15')
Output Type:
  TensorSpec(shape=(None, 37), dtype=tf.float32, name=None)
Captures:
  139519495819808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139519495824912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139519495819984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139519495826320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139519495821392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139519495824384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139519495828608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139519495829312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139519495822976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139519495966384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139519495965856

In [ ]:
pip install openvino==2023.3.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 14.7 MB/s eta 0:00:00


In [ ]:
import openvino as ov